<a href="https://colab.research.google.com/github/cricci3/Salaries_clustering/blob/main/Project2_DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
'''
Data Analytics
Assignment 2 - Group 2
Salaries (clustering)

Students: Pernille Bergesen (pernille.bergesen@usi.ch) and Claudio Ricci (riccicl@usi.ch)
'''

'\nData Analytics\nAssignment 2 - Group 2\nSalaries (clustering)\n\nStudents: Pernille Bergesen (pernille.bergesen@usi.ch) and Claudio Ricci (riccicl@usi.ch)\n'

In [3]:
import torch
import pandas as pd
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#loading the data from the san-francisco datasets
dataset_file_2011 = '/content/drive/MyDrive/DataAnalytics_02_Salaries/san-francisco-2011.csv'
dataset_file_2012 = '/content/drive/MyDrive/DataAnalytics_02_Salaries/san-francisco-2012.csv'
dataset_file_2013 = '/content/drive/MyDrive/DataAnalytics_02_Salaries/san-francisco-2013.csv'
dataset_file_2014 = '/content/drive/MyDrive/DataAnalytics_02_Salaries/san-francisco-2014.csv'

#just for testing:
#printing first two lines of 2011 dataset

df_2011 = pd.read_csv(dataset_file_2011)
df_2012 = pd.read_csv(dataset_file_2012)
df_2013 = pd.read_csv(dataset_file_2013)
df_2014 = pd.read_csv(dataset_file_2014)


print("san-francisco-2011.csv: ")
df_2011.head(2)

san-francisco-2011.csv: 


,Employee Name,Job Title,Base Pay,Overtime Pay,Other Pay,Benefits,Total Pay,Total Pay & Benefits,Year,Notes,Agency
0,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.00,400184.25,Not Provided,567595.43,567595.43,2011,NaN,San Francisco
1,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,Not Provided,538909.28,538909.28,2011,NaN,San Francisco


In [5]:
#printing first two lines of 2012 dataset
print("san-francisco-2012.csv: ")
df_2012.head(2)

san-francisco-2012.csv: 


,Employee Name,Job Title,Base Pay,Overtime Pay,Other Pay,Benefits,Total Pay,Total Pay & Benefits,Year,Notes,Agency
0,Gary Altenberg,"Lieutenant, Fire Suppression",128808.87,220909.48,13126.31,44430.12,362844.66,407274.78,2012,NaN,San Francisco
1,Gregory Suhr,Chief of Police,302578.00,0.00,18974.11,69810.19,321552.11,391362.30,2012,NaN,San Francisco


In [6]:
#printing first two lines of 2013 dataset
print("san-francisco-2013.csv: ")
df_2013.head(2)

san-francisco-2013.csv: 


,employee_name,job_title,base_pay,overtime_pay,other_pay,total_benefits,total_pay,total_pay_benefits,year,notes,jurisdiction_name
0,Gregory P Suhr,Chief of Police,319275.01,0.0,20007.06,86533.21,339282.07,425815.28,2013,NaN,San Francisco
1,Joanne M Hayes-White,"Chief, Fire Department",313686.01,0.0,23236.00,85431.39,336922.01,422353.40,2013,NaN,San Francisco


In [7]:
#printing first two lines of 2014 dataset
print("san-francisco-2014.csv: ")
df_2014.head(2)

san-francisco-2014.csv: 


,Employee Name,Job Title,Base Pay,Overtime Pay,Other Pay,Benefits,Total Pay,Total Pay & Benefits,Year,Notes,Agency,Status
0,David Shinn,Deputy Chief 3,129150.01,0.00,342802.63,38780.04,471952.64,510732.68,2014,NaN,San Francisco,PT
1,Amy P Hart,Asst Med Examiner,318835.49,10712.95,60563.54,89540.23,390111.98,479652.21,2014,NaN,San Francisco,FT


In [8]:
df_2011.columns

Index(['Employee Name', 'Job Title', 'Base Pay', 'Overtime Pay', 'Other Pay',
       'Benefits', 'Total Pay', 'Total Pay & Benefits', 'Year', 'Notes',
       'Agency'],
      dtype='object')

In [9]:
df_2012.columns

Index(['Employee Name', 'Job Title', 'Base Pay', 'Overtime Pay', 'Other Pay',
       'Benefits', 'Total Pay', 'Total Pay & Benefits', 'Year', 'Notes',
       'Agency'],
      dtype='object')

In [10]:
df_2013.columns

Index(['employee_name', 'job_title', 'base_pay', 'overtime_pay', 'other_pay',
       'total_benefits', 'total_pay', 'total_pay_benefits', 'year', 'notes',
       'jurisdiction_name'],
      dtype='object')

In [11]:
df_2014.columns

Index(['Employee Name', 'Job Title', 'Base Pay', 'Overtime Pay', 'Other Pay',
       'Benefits', 'Total Pay', 'Total Pay & Benefits', 'Year', 'Notes',
       'Agency', 'Status'],
      dtype='object')

df2011/12/14 have column Agency while df2013 has column jurisdiction_name but both have just a class inside, 'San Francisco' -> I will rename 'jurisdiction_name' in 'Agency'

In [12]:
df_2011['Agency'].unique()
df_2012['Agency'].unique()
df_2013['jurisdiction_name'].unique()
df_2014['Agency'].unique()

array(['San Francisco'], dtype=object)

In [13]:
df_2013.rename(columns={'jurisdiction_name': 'Agency'}, inplace=True)

# others rename
df_2013.rename(columns={'employee_name' : 'Employee Name',
                        'job_title' : 'Job Title',
                        'base_pay' : 'Base Pay',
                        'overtime_pay' : 'Overtime Pay',
                        'other_pay' : 'Other Pay',
                        'total_benefits' : 'Benefits',
                        'total_pay' : 'Total Pay',
                        'total_pay_benefits' : 'Total Pay & Benefits',
                        'year' : 'Year',
                        'notes' : 'Notes'}, inplace=True)

In [14]:
df_2013.columns

Index(['Employee Name', 'Job Title', 'Base Pay', 'Overtime Pay', 'Other Pay',
       'Benefits', 'Total Pay', 'Total Pay & Benefits', 'Year', 'Notes',
       'Agency'],
      dtype='object')

In [15]:
# Df_2014 has also status : Part-time / Full-time / nan -> I will delete it so we can merge the four datasets
df_2014['Status'].unique()

array(['PT', 'FT', nan], dtype=object)

In [16]:
df_2014.drop(columns=['Status'], inplace=True)

In [17]:
df_2014.columns

Index(['Employee Name', 'Job Title', 'Base Pay', 'Overtime Pay', 'Other Pay',
       'Benefits', 'Total Pay', 'Total Pay & Benefits', 'Year', 'Notes',
       'Agency'],
      dtype='object')

In [18]:
df_2014['Year'].unique()

array([2014])

In [19]:
df = pd.concat([df_2011, df_2012, df_2013, df_2014])

In [20]:
df.head(5)

,Employee Name,Job Title,Base Pay,Overtime Pay,Other Pay,Benefits,Total Pay,Total Pay & Benefits,Year,Notes,Agency
0,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.0,400184.25,Not Provided,567595.43,567595.43,2011,NaN,San Francisco
1,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,Not Provided,538909.28,538909.28,2011,NaN,San Francisco
2,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.6,Not Provided,335279.91,335279.91,2011,NaN,San Francisco
3,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.0,56120.71,198306.9,Not Provided,332343.61,332343.61,2011,NaN,San Francisco
4,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6,9737.0,182234.59,Not Provided,326373.19,326373.19,2011,NaN,San Francisco


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 148654 entries, 0 to 38122
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Employee Name         148654 non-null  object 
 1   Job Title             148654 non-null  object 
 2   Base Pay              148049 non-null  object 
 3   Overtime Pay          148654 non-null  object 
 4   Other Pay             148654 non-null  object 
 5   Benefits              148654 non-null  object 
 6   Total Pay             148654 non-null  float64
 7   Total Pay & Benefits  148654 non-null  float64
 8   Year                  148654 non-null  int64  
 9   Notes                 0 non-null       float64
 10  Agency                148654 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 13.6+ MB


In [22]:
df.isnull().sum()

,0
Employee Name,0
Job Title,0
Base Pay,605
Overtime Pay,0
Other Pay,0
Benefits,0
Total Pay,0
Total Pay & Benefits,0
Year,0
Notes,148654


In [23]:
# Notes columns if full of null values -> drop
df.drop(columns=['Notes'], inplace=True)

In [24]:
# I will also remove Name (useless)
df.drop(columns=['Employee Name'], inplace=True)

In [25]:
# Convert 'Pay' to numeric, coercing non-numeric values to NaN
df['Overtime Pay'] = pd.to_numeric(df['Overtime Pay'], errors='coerce')
df['Other Pay'] = pd.to_numeric(df['Other Pay'], errors='coerce')
df['Benefits'] = pd.to_numeric(df['Benefits'], errors='coerce')
df['Base Pay'] = pd.to_numeric(df['Base Pay'], errors='coerce')

In [26]:
df.isnull().sum()

,0
Job Title,0
Base Pay,609
Overtime Pay,4
Other Pay,4
Benefits,36163
Total Pay,0
Total Pay & Benefits,0
Year,0
Agency,0


In [28]:
# Check how many NaNs we have now
print(f"Number of NaN values in Base Pay: {df['Base Pay'].isna().sum()}")

# Now fill NaN values with median
df['Base Pay'].fillna(df['Base Pay'].median(), inplace=True)
df['Overtime Pay'].fillna(df['Overtime Pay'].median(), inplace=True)
df['Other Pay'].fillna(df['Other Pay'].median(), inplace=True)
df['Benefits'].fillna(df['Benefits'].median(), inplace=True)

# Verify all NaNs are gone
print(f"Number of NaN values in Base Pay after manipulations: {df.isnull().sum()}")

Number of NaN values in Base Pay: 0
Number of NaN values in Base Pay after manipulations: Job Title               0
Base Pay                0
Overtime Pay            0
Other Pay               0
Benefits                0
Total Pay               0
Total Pay & Benefits    0
Year                    0
Agency                  0
dtype: int64


In [37]:
df.isnull().sum()

,0
Job Title,0
Base Pay,0
Overtime Pay,4
Other Pay,4
Benefits,36163
Total Pay,0
Total Pay & Benefits,0
Year,0
Agency,0


In [ ]:
#Use at least two different clustering algorithms and compare them against one another.
#Testing to see which cluster algorithm is best for the
'''
Firstly, cluster for the jobs based on their various attributes, make it as categories of jobs based on salaries and benefits.
Then eventually, cluster for employeees as well based on their job conditions.
'''

#K-means clustering:
'''
blobs = datasets.make_blobs(n_samples=600, centers=4,
                            cluster_std=0.60, random_state=seed)
moons = datasets.make_moons(600, noise=.05, random_state=seed)

fig, ax = plt.subplots(1, 2, figsize=(8, 3.5), dpi=150)
ax[0].scatter(blobs[0][:, 0], blobs[0][:, 1], edgecolor='none', facecolor='red', s=35, alpha=.5)
ax[0].axis('equal')
ax[1].scatter(moons[0][:, 0], moons[0][:, 1], edgecolor='none', facecolor='red', s=35, alpha=.5)
ax[1].axis('equal')
'''

#Hierarchical Clustering:
#Agglomerative (bottom up):
#They are divided into agglomerative and divisive algorithms
